In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

In [18]:
N = 10  # 데이터 갯수
K = 4   # 범주 갯수
mu_0 = np.ones(K) / K   # np.ones : 모든 요소의 값이 1인 ndarray 만들기 -> 발생 확률(임의로 만든것)
# (K개=4개) 각 1, 4개가 4로 나뉘어 0.25 4개 됨
print(mu_0)

np.random.seed(42)
x = np.random.choice(K, N, p=mu_0)
# 랜덤 시드 - 재현성 - 난수가 발생해도 같은 난수가 발생하게 하는 것
# 컴퓨터는 뽑을때 규칙적이라 뽑힐 수를 흔들어 놓아서 난수 발생되게 한다
# 기본 시드 값을 현재 시간으로 해서 매번 달라지는 것

# x = np.random.choice(K, N, p=mu_0)
# K범주 안에서 (4개) N개(10개)를 뽑고, p=mu_0는 확률(25%)
# np.random.choice 구글링 - 넘파이 공식 문서 보기
# https://numpy.org/doc/2.1/reference/random/generated/numpy.random.choice.html

print(x)

c = np.bincount(x)
print(c)
# array([3, 1, 4, 2]) > 0이 3개, 1이 1개, 2가 4개, 3이 2개 라는 뜻

print(stats.chisquare(c))
print(stats.chisquare([7, 1, 1, 1]))
# pvalue=np.float64(0.5009344477491052)
# 각 동일 분표일때 불규칙 샘플이 뽑힐 확률
# 샘플이 같아도 50% 매우 빈번하다

[0.25 0.25 0.25 0.25]
[1 3 2 2 0 0 0 3 2 2]
[3 1 4 2]
Power_divergenceResult(statistic=np.float64(2.0), pvalue=np.float64(0.5724067044708798))
Power_divergenceResult(statistic=np.float64(10.8), pvalue=np.float64(0.012858001156297613))
